# An Intro to Arvo (and bits of music21)

This is mostly me going through the arvo [intro video](https://www.youtube.com/watch?v=qxB7r4hnEL8&t=23s).

I also pulled a few ideas some other resources, for example
- [this intro to music21, which I ought to go through fully later](https://opencomputinglab.github.io/SubjectMatterNotebooks/music/overview.html).
- [this page in the music21 docs](https://web.mit.edu/music21/doc/usersGuide/usersGuide_02_notes.html)


In [ ]:
import subprocess

from music21 import environment
from arvo import tools

user_settings = environment.UserSettings()

# Find musescore provided via nix flakes
MUSESCORE_EXE = subprocess.check_output(["which", "mscore."]).strip().decode()
MUSESCORE_DIR = MUSESCORE_EXE.removesuffix('/bin/mscore.')
MUSESCORE_APP = MUSESCORE_DIR + "/Applications/mscore.app/"
user_settings["musicxmlPath"] = MUSESCORE_APP
user_settings["musescoreDirectPNGPath"] = MUSESCORE_EXE

# Find lilypond provided via nix flakes
LILYPOND_EXE = subprocess.check_output(["which", "lilypond"]).strip().decode()
LILYPOND_VERSION = subprocess.check_output(["lilypond", "--version"]).strip().decode().split()[2]
user_settings["lilypondPath"] = LILYPOND_EXE
user_settings["lilypondVersion"] = LILYPOND_VERSION

dict(user_settings)


In [ ]:
# (By default, notes are in octave 4)
melody = tools.notes_to_stream(["C4", "D4", "Eb", "E", "F", "G", "Bb", "C5", "D"])

# default show() will open it inline, although the format is a bit ugly (it sizes for a full page of music!)
melody.show()

In [ ]:
from arvo import minimalism, isorhythm

# Same melody as before
melody = tools.notes_to_stream(["C4", "D4", "Eb", "E", "F", "G", "Bb", "C5", "D"])

# recombine chunks of the melody into a longer stream
melody_1 = minimalism.additive_process(melody)

# redefine the rhythmic behavior via isorhythm
durations = tools.durations_to_stream([2, 1, .5, .5, 1, .5, .5, .5, 1, .5])  # (here 1 is a quarter note)
melody_2 = isorhythm.create_isorhythm(melody_1, durations)

melody_2.show(fmt="musicxml")

In [ ]:
# Do it all over again, using a backward additive process with more repetition.
# ... There are a few other options you can try for `direction`

melody = tools.notes_to_stream(["C4", "D4", "Eb", "E", "F", "G", "Bb", "C5", "D"])
melody_1 = minimalism.additive_process(melody, direction=minimalism.Direction.BACKWARD, repetitions=2)
durations = tools.durations_to_stream([2, 1, .5, .5, 1, .5, .5, .5, 1, .5])  # (here 1 is a quarter note)
melody_2 = isorhythm.create_isorhythm(melody_1, durations)
melody_2.show()

# NOTE: I ought to submit a PR on arvo to fix the call to .flat!

In [ ]:
# Midi playback avoids putting musescore in the loop
#   You can get it natively in python - handy for little snippets - or export to a real synth
#   Obvious options for a synth: Supercollider, Ableton

from music21 import midi

# So far unfortunately I haven't figured out how to *stop* playback, so be careful with this!
#   It's one reason to prefer the musescore / musicxml approach for longer melodies (melody_2 takes a while)
sound = midi.realtime.StreamPlayer(melody)
sound.play()

# You could run this path through any other midi player as an alternative:
melody.write(fmt="midi")

In [ ]:
# You can get the ability to interrupt playback by using IPython midi player:
melody.show(fmt="midi")

In [ ]:
# Unfortunately this doesn't work :(
'''
path = melody.write(fmt="vexflow")
with open(path) as f:
    html = f.read()

from IPython.core.display import HTML
HTML("<iframe>" + html + "</iframe>")

from IPython.display import IFrame
IFrame(src="file://" + str(path), width=700, height=600)
'''

In [ ]:
from music21 import converter

# Another way to insert music is using tinyNotation

melody = converter.parse('tinyNotation: 4/4 C4 D2 E4 F4 G4 A4 B4 c4')
melody.show(fmt="midi")

In [ ]:
# Another demo from the video: retrograde
from arvo import transformations

melody = tools.notes_to_stream(["C4", "D4", "Eb", "E", "F", "G", "Bb", "C5", "D"])
melody_1 = melody  # copy for mutation
melody_1.append(transformations.retrograde(melody))
melody_1.show(fmt="text")

In [ ]:
# need to read up on what this really is...
from arvo import tintinnabuli

melody = tools.notes_to_stream(["C4", "D4", "Eb", "E", "F", "G", "Bb", "C5", "D"])
melody_1 = tintinnabuli.create_t_voice(
    melody,
    ["C", "Eb", "G"],
    position=2,
    direction=tintinnabuli.Direction.DOWN,
)
melody_1.show(fmt="text")

# ... by the end of the video, he actually has something that reminds me
# of a really cool TidalCycles loop rather than a music theory thing which is interesting.

In [ ]:
# Sadly I think there's a version mismatch or something, lilypond support is broken
# as of my current pipfile and flake locks; setting the version didn't seem to help :/
'''
melody.show("lilypond")
'''
# This could be worth looking into eventually (maybe some OSS contributions) but for now
# honestly the built-in show/midi + the musescore musicxml export are probably better anyway